In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import RandomOverSampler
import cleanlab
import pickle

In [ ]:
df=pd.read_csv(r"C:\Users\tareq\Desktop\All_Users_Labeled.csv")

In [ ]:
df.drop(['Unnamed: 0','Username'],axis=1,inplace=True)

In [ ]:
y = df['o'].values

In [ ]:
X_num = df.iloc[:, 1:5].values
ss=MinMaxScaler()
X_num=ss.fit_transform(X=X_num)
vectorizer = CountVectorizer()
X_text = vectorizer.fit_transform(df['Full_Text'])
df1=pd.DataFrame(X_text.toarray(),columns=vectorizer.get_feature_names_out())
df1["total_words"]= [x[0] for x in X_num]
df1["total_char"]= [x[1] for x in X_num]
df1["avg_chr_per_word"]= [x[2] for x in X_num]
df1["total_emojis"]= [x[3] for x in X_num]

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(df1, y, test_size=0.2, random_state=42)

In [ ]:
s=RandomOverSampler()
X_train_r, y_train_r=s.fit_resample(X_train,y_train)

In [ ]:
rfc_parameters = {'class_weight': ['balanced_subsample','balanced',None], 'criterion': ['entropy','log_loss','gini'],'max_features':['sqrt','log2',None],'n_estimators':[10,100,500,1000,1500],'max_depth':[10,50,125,170,200],'random_state':[42]}
rfc = RandomForestClassifier()
rfc_grid = GridSearchCV(rfc, rfc_parameters, cv=5)
rfc_grid.fit(X_train_r, y_train_r)

In [ ]:
m=rfc_grid.best_estimator_
m.fit(X_train_r,y_train_r)

In [ ]:
y_pred=m.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.84      0.87       247
           1       0.61      0.75      0.67        83

    accuracy                           0.82       330
   macro avg       0.76      0.79      0.77       330
weighted avg       0.83      0.82      0.82       330

In [ ]:
pickle.dump(m, open('final_ml_models/O.pkl', 'wb'))